In [1]:
!pip install -q findspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 49.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=2462a54ebc3007fc5bc1437feab9bb37222a4c8336db3717297c76b98255f4dc
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sc.version

'3.3.1'

## SVM

In [8]:
from pyspark.ml.classification import LinearSVC

# load training data

training = spark.read.format("libsvm").load("/content/sample_libsvm_data.txt")
lsvc = LinearSVC(maxIter=10, regParam=0.1)

# fit the model

lsvcModel = lsvc.fit(training)

# print the coefficients and intercept for linear SVC

print("coefficients: ", str(lsvcModel.coefficients))
print("intercept: " + str(lsvcModel.intercept))

coefficients:  [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00015154081891400172,-3.4357432628275696e-05,6.8868723771654e-05,0.0005825396368790324,0.0002658667437974877,-5.4448990232205866e-06,-0.000410876298911309,-0.00023771334401618933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0015051243197375345,0.0005056741785679464,0.0007739871946118443,-7.439317729362511e-05,2.2395429551533054e-07,2.1502767568913162e-05,4.000155795906807e-05,2.841045988826015e-05,1.2172703998609027e-05,-1.4702408529921009e-05,-4.00596456869388e-05,3.0693747761902103e-06,0.00015395475863074347,0.00015205858963404883,-0.00021785419667457335,0.0,0.0,0.0,0.0,0

In [10]:
training.show(50, truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
#from pyspark import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from sklearn.metrics import confusion_matrix
from sklearn.datasets import load_breast_cancer
import pandas as pd

In [16]:
bc = load_breast_cancer()

df_bc = pd.DataFrame(bc.data, columns=bc.feature_names)
df_bc['label'] = pd.Series(bc.target)

In [19]:
df_bc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [21]:
sqlContext = SQLContext(sc)

data = sqlContext.createDataFrame(df_bc)
print(data.printSchema())

root
 |-- mean radius: double (nullable = true)
 |-- mean texture: double (nullable = true)
 |-- mean perimeter: double (nullable = true)
 |-- mean area: double (nullable = true)
 |-- mean smoothness: double (nullable = true)
 |-- mean compactness: double (nullable = true)
 |-- mean concavity: double (nullable = true)
 |-- mean concave points: double (nullable = true)
 |-- mean symmetry: double (nullable = true)
 |-- mean fractal dimension: double (nullable = true)
 |-- radius error: double (nullable = true)
 |-- texture error: double (nullable = true)
 |-- perimeter error: double (nullable = true)
 |-- area error: double (nullable = true)
 |-- smoothness error: double (nullable = true)
 |-- compactness error: double (nullable = true)
 |-- concavity error: double (nullable = true)
 |-- concave points error: double (nullable = true)
 |-- symmetry error: double (nullable = true)
 |-- fractal dimension error: double (nullable = true)
 |-- worst radius: double (nullable = true)
 |-- worst 

In [23]:
features = bc.feature_names

va = VectorAssembler(inputCols = features, outputCol='features');

va_df = va.transform(data)
va_df = va_df.select(['features', 'label'])
va_df.show(50)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[17.99,10.38,122....|    0|
|[20.57,17.77,132....|    0|
|[19.69,21.25,130....|    0|
|[11.42,20.38,77.5...|    0|
|[20.29,14.34,135....|    0|
|[12.45,15.7,82.57...|    0|
|[18.25,19.98,119....|    0|
|[13.71,20.83,90.2...|    0|
|[13.0,21.82,87.5,...|    0|
|[12.46,24.04,83.9...|    0|
|[16.02,23.24,102....|    0|
|[15.78,17.89,103....|    0|
|[19.17,24.8,132.4...|    0|
|[15.85,23.95,103....|    0|
|[13.73,22.61,93.6...|    0|
|[14.54,27.54,96.7...|    0|
|[14.68,20.13,94.7...|    0|
|[16.13,20.68,108....|    0|
|[19.81,22.15,130....|    0|
|[13.54,14.36,87.4...|    1|
|[13.08,15.71,85.6...|    1|
|[9.504,12.44,60.3...|    1|
|[15.34,14.26,102....|    0|
|[21.16,23.04,137....|    0|
|[16.65,21.38,110....|    0|
|[17.14,16.4,116.0...|    0|
|[14.58,21.53,97.4...|    0|
|[18.61,20.25,122....|    0|
|[15.3,25.27,102.4...|    0|
|[17.57,15.05,115....|    0|
|[18.63,25.11,124....|    0|
|[11.84,18.7,7

In [24]:
(train, test) = va_df.randomSplit([0.9, 0.1])

lsvc = LinearSVC(labelCol="label", maxIter=50)
lsvc = lsvc.fit(train)

In [25]:
pred = lsvc.transform(test)
pred.show(20)

+--------------------+-----+--------------------+----------+
|            features|label|       rawPrediction|prediction|
+--------------------+-----+--------------------+----------+
|[10.2,17.48,65.05...|    1|[-10.181856418604...|       1.0|
|[10.49,19.29,67.4...|    1|[-9.1674250086436...|       1.0|
|[11.22,33.81,70.7...|    1|[-6.1778960675417...|       1.0|
|[11.29,13.04,72.2...|    1|[-8.7288810342486...|       1.0|
|[11.3,18.19,73.93...|    1|[-3.9678309683516...|       1.0|
|[11.31,19.04,71.8...|    1|[-6.5450155779848...|       1.0|
|[11.41,14.92,73.5...|    1|[-9.6993506355405...|       1.0|
|[11.52,14.93,73.8...|    1|[-6.4474492083288...|       1.0|
|[11.6,12.84,74.34...|    1|[-7.0407098425462...|       1.0|
|[11.8,16.58,78.99...|    0|[-1.3408405163339...|       1.0|
|[12.31,16.52,79.1...|    1|[-6.5902989004573...|       1.0|
|[12.34,22.22,79.8...|    1|[-4.7656702088372...|       1.0|
|[12.62,23.97,81.3...|    1|[-2.6637909021969...|       1.0|
|[12.77,22.47,81.7...|  

In [26]:
evaluator=MulticlassClassificationEvaluator(metricName="accuracy")
acc = evaluator.evaluate(pred)
print("Prediction Accuracy: ", acc)


y_pred= pred.select("prediction").collect()
y_orig= pred.select("label").collect()

cm = confusion_matrix(y_orig, y_pred)
print("confusion Matrix: ")
print(cm)

Prediction Accuracy:  0.9090909090909091
confusion Matrix: 
[[17  5]
 [ 0 33]]
